<a href="https://colab.research.google.com/github/Cerabbite/CGEL/blob/main/Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For each word in the top 10.000 most used English, Spanish, Japanese and Chinese words. There will be 100-1.000 sample sentences with those words (written in using the english alphabet). Using these sentences this script is going to work out how related words are.

In [2]:
# Import libraries
#!pip install keras
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [33]:
tokenizer = Tokenizer()

data = open("English sentences Dataset.txt").read()
corpus = data.lower().split("\n")

avg_list = []
for i in corpus:
  avg_list.append(len(i.split(" ")))

print(sum(avg_list)/len(avg_list))
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

4.32687338501292


In [25]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [26]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [27]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [28]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=20, verbose=1)


Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


81/81 [==============================] - 54s 41ms/step - loss: 5.6541 - accuracy: 0.0819
Epoch 2/20
81/81 [==============================] - 3s 41ms/step - loss: 4.6012 - accuracy: 0.1696
Epoch 3/20
81/81 [==============================] - 3s 41ms/step - loss: 3.4709 - accuracy: 0.2920
Epoch 4/20
81/81 [==============================] - 3s 41ms/step - loss: 2.4780 - accuracy: 0.4276
Epoch 5/20
81/81 [==============================] - 3s 40ms/step - loss: 1.8421 - accuracy: 0.5388
Epoch 6/20
81/81 [==============================] - 3s 39ms/step - loss: 1.4205 - accuracy: 0.6311
Epoch 7/20
81/81 [==============================] - 3s 40ms/step - loss: 1.2191 - accuracy: 0.6775
Epoch 8/20
81/81 [==============================] - 3s 40ms/step - loss: 1.0883 - accuracy: 0.6895
Epoch 9/20
81/81 [==============================] - 3s 40ms/step - loss: 1.0417 - accuracy: 0.6933
Epoch 10/20
81/81 [==============================] - 3s 41ms/step - loss: 0.9936 - accuracy: 0.6972
Epoch 11/20
81/81 [

In [39]:
# Test the model
seed_text = ""
next_words = 5

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = model.predict(token_list, verbose=0)
  classes = np.argmax(predicted,axis=1)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == classes:
      output_word = word
      break
  seed_text += f" {output_word}"

print(seed_text)


 is everything alright fact the
